In [1]:
import keras
from IPython.display import Image

Using TensorFlow backend.


In [2]:
import os
import random
import re
from PIL import Image

# Rozdelenie snimkov na train, val a test

In [3]:
DATA_PATH = 'C:\Dataset\\'
FRAME_PATH = DATA_PATH +'Snimky\\'
MASK_PATH = DATA_PATH +'Mask\\'


In [4]:
# Create folders to hold images and masks

folders = ['train_frames\\train', 'train_masks\\train', 'val_frames\\val', 'val_masks\\val', 'test_frames\\test', 'test_masks\\test']


for folder in folders:
    os.makedirs(DATA_PATH + folder)

In [5]:
all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)

In [6]:
all_frames.sort(key=lambda var:[int(x) if x.isdigit() else x 
                                for x in re.findall(r'[^0-9]|[0-9]+', var)])
all_masks.sort(key=lambda var:[int(x) if x.isdigit() else x 
                               for x in re.findall(r'[^0-9]|[0-9]+', var)])

In [217]:
#  NEROBIT TOTO LEBO TO PREMIESA A POTOM TO NENAMAPUJE PRESNE TAK AKO CHCEME
# random.seed(21)
# random.shuffle(all_frames)

In [7]:
# Generate train, val, and test sets for frames

train_split = int(0.7*len(all_frames))
val_split = int(0.9 * len(all_frames))

train_frames = all_frames[:train_split]
val_frames = all_frames[train_split:val_split]
test_frames = all_frames[val_split:]

In [8]:
# Generate corresponding mask lists for masks

train_masks = [f for f in all_masks if f in train_frames]
val_masks = [f for f in all_masks if f in val_frames]
test_masks = [f for f in all_masks if f in test_frames]

In [9]:
def add_frames(dir_name, image):
  
  img = Image.open(FRAME_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'\\'+image)
  
  
  
def add_masks(dir_name, image):
  
  img = Image.open(MASK_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'\\'+image)

In [10]:
frame_folders = [(train_frames, 'train_frames\\train'), (val_frames, 'val_frames\\val'), 
                 (test_frames, 'test_frames\\test')]

mask_folders = [(train_masks, 'train_masks\\train'), (val_masks, 'val_masks\\val'), 
                (test_masks, 'test_masks\\test')]


In [11]:
for folder in frame_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)

  list(map(add_frames, name, array))

In [12]:
for folder in mask_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)
  
  list(map(add_masks, name, array))

# Vytvorenie generatoru

## Potrebna organizacia priecinkov

![dir_organisation](priecinky.jpg)

## definovanie generatora

In [13]:
from keras.preprocessing.image import ImageDataGenerator

In [14]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

In [15]:
train_image_generator = train_datagen.flow_from_directory(
'C:/Dataset/train_frames',
    target_size = (320,320),
    class_mode= None,
    batch_size = 2)

train_mask_generator = train_datagen.flow_from_directory(
'C:/Dataset/train_masks',
    target_size = (320,320),
    class_mode= None,
    color_mode = 'grayscale',
batch_size = 2)


val_image_generator = val_datagen.flow_from_directory(
'C:/Dataset/val_frames',
    target_size = (320,320),
    class_mode= None,
    batch_size = 2)


val_mask_generator = val_datagen.flow_from_directory(
'C:/Dataset/val_masks',
    target_size = (320,320),
    class_mode= None,
    color_mode = 'grayscale',
batch_size = 2)



train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 1750 images belonging to 1 classes.
Found 1750 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.


# Skuska generatoru

**Nacitanie modelu**

In [16]:
from src.Model import get_unet

In [17]:
input_img = keras.layers.Input(shape=(320,320,3))
model = get_unet(input_img)

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 320, 320, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 320, 320, 16) 64          conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 320, 320, 16) 0           batch_normalization_2[0][0]      
____________________________________________________________________________________________

**Trenovanie**

In [19]:
optimizer = keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['accuracy'],
              #sample_weight_mode='temporal'
              )

In [21]:
model.fit_generator(generator=train_generator,epochs=2,steps_per_epoch=5)

Epoch 1/2
5/5 [==============================] - 10s 2s/step - loss: 8756.0180 - accuracy: 0.4556
Epoch 2/2
5/5 [==============================] - 10s 2s/step - loss: 5527.2706 - accuracy: 0.5126


# ulozenie modelu

In [ ]:
model.save('Model.h5')